In [1]:
!pip install transformers
!pip install torch
!pip install streamlit
!pip install --upgrade accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    pipeline
)
import torch
import random
import textwrap


In [3]:
def generate_story(prompt, model_name, max_length=300, temperature=0.8, top_k=50, top_p=0.95):
    if model_name == "gpt2":
        tokenizer = AutoTokenizer.from_pretrained("gpt2")
        model = AutoModelForCausalLM.from_pretrained("gpt2")
    elif model_name == "facebook/bart-large":
        tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
        model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")
    elif model_name == "google/flan-t5-large":
        tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
        model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
    else:
        raise ValueError("Unsupported model name!")

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    output_ids = model.generate(
        input_ids,
        do_sample=True,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    story = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return "\n".join(textwrap.wrap(story, width=100))


In [4]:
def generate_story(prompt, model_name, max_length=300, temperature=0.8, top_k=50, top_p=0.95):
    if model_name == "gpt2":
        tokenizer = AutoTokenizer.from_pretrained("gpt2")
        model = AutoModelForCausalLM.from_pretrained("gpt2")
        input_ids = tokenizer.encode(prompt, return_tensors="pt")

        output_ids = model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )

        story = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    elif model_name == "facebook/bart-large":
        tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
        model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")
        # BART needs a prompt like a title or input text to expand
        bart_prompt = f"Generate a story: {prompt}"

        input_ids = tokenizer(bart_prompt, return_tensors="pt", truncation=True).input_ids

        output_ids = model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            num_return_sequences=1
        )

        story = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    elif model_name == "google/flan-t5-large":
        tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
        model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
        # FLAN-T5 is instruction-tuned, so we give an instruction-style prompt
        t5_prompt = f"Tell a story about: {prompt}"

        input_ids = tokenizer(t5_prompt, return_tensors="pt", truncation=True).input_ids

        output_ids = model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            num_return_sequences=1
        )

        story = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    else:
        raise ValueError("Unsupported model name!")

    return "\n".join(textwrap.wrap(story, width=100))


In [5]:
def generate_story(prompt, model_name, max_length=300, temperature=0.8, top_k=50, top_p=0.95):
    if model_name == "gpt2":
        tokenizer = AutoTokenizer.from_pretrained("gpt2")
        model = AutoModelForCausalLM.from_pretrained("gpt2")
        input_ids = tokenizer.encode(prompt, return_tensors="pt")

        output_ids = model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )

        story = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    elif model_name == "EleutherAI/gpt-neo-1.3B":
        tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
        model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids

        output_ids = model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )

        story = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    elif model_name == "facebook/bart-large":
        tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
        model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")
        bart_prompt = f"Generate a story: {prompt}"
        input_ids = tokenizer(bart_prompt, return_tensors="pt", truncation=True).input_ids

        output_ids = model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
        )

        story = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    elif model_name == "google/flan-t5-large":
        tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
        model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
        t5_prompt = f"Tell a story about: {prompt}"
        input_ids = tokenizer(t5_prompt, return_tensors="pt", truncation=True).input_ids

        output_ids = model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
        )

        story = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    else:
        raise ValueError("Unsupported model name!")

    return "\n".join(textwrap.wrap(story, width=100))


In [6]:
prompt = "Once upon a time in a magical forest, there lived a curious fox who dreamed of flying."

# GPT-2 Story
print("📘 GPT-2 Generated Story:\n")
print(generate_story(prompt, model_name="gpt2"))

# BART Story
print("\n📗 BART Generated Story:\n")
print(generate_story(prompt, model_name="facebook/bart-large"))

# FLAN-T5 Story
print("\n📕 FLAN-T5 Generated Story:\n")
print(generate_story("Tell a story: " + prompt, model_name="google/flan-t5-large"))


📘 GPT-2 Generated Story:



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

When the fox was a little boy, he had dreams of becoming a fox. One day, a fox woke up and found
himself in a magical woods. The fox decided to fly. The fox took off and landed at the edge of the
magical woods.


In [7]:
pip install gradio transformers torch

In [9]:
import gradio as gr
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
)
import textwrap

def generate_story(prompt, model_name, max_length=300, temperature=0.8, top_k=50, top_p=0.95):
    if model_name == "gpt2":
        tokenizer = AutoTokenizer.from_pretrained("gpt2")
        model = AutoModelForCausalLM.from_pretrained("gpt2")
        input_ids = tokenizer.encode(prompt, return_tensors="pt")

        output_ids = model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )
        story = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    elif model_name == "EleutherAI/gpt-neo-1.3B":
        tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
        model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B")
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids

        output_ids = model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id
        )
        story = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    elif model_name == "facebook/bart-large":
        tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
        model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large")
        input_ids = tokenizer("Generate a story: " + prompt, return_tensors="pt", truncation=True).input_ids

        output_ids = model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
        )
        story = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    elif model_name == "google/flan-t5-large":
        tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")
        model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")
        input_ids = tokenizer("Tell a story about: " + prompt, return_tensors="pt", truncation=True).input_ids

        output_ids = model.generate(
            input_ids,
            do_sample=True,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
        )
        story = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    else:
        return "❌ Unsupported model!"

    return "\n".join(textwrap.wrap(story, width=100))


# Gradio Interface
gr.Interface(
    fn=generate_story,
    inputs=[
        gr.Textbox(label="Story Prompt", lines=4, placeholder="Enter a story prompt..."),
        gr.Radio(
            ["gpt2", "EleutherAI/gpt-neo-1.3B", "facebook/bart-large", "google/flan-t5-large"],
            label="Choose Model",
            value="gpt2"
        ),
        gr.Slider(100, 1024, value=300, step=10, label="Max Length"),
        gr.Slider(0.1, 1.5, value=0.8, step=0.1, label="Temperature"),
        gr.Slider(0, 100, value=50, step=5, label="Top-K"),
        gr.Slider(0.5, 1.0, value=0.95, step=0.05, label="Top-P"),
    ],
    outputs=gr.Textbox(label="Generated Story", lines=20),
    title="📖 Story Generator with Transformers",
    description="Enter a prompt and generate a story using one of several transformer models."
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://695d1214d21b6726fe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
